In [1]:
#Importamos las librerias necesarias para crear la red neuronal
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf
#Leemos el csv
evaluation = pandas.read_csv('turkiye-student-evaluation_R_Specific.csv', header=0)

In [2]:
#Mostramos las 10 primeras lineas para ver que se ha leido bien
evaluation.head(10)

,instr,class,nb.repeat,attendance,difficulty,Q1,Q2,Q3,Q4,Q5,...,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27,Q28
1,1,2,1,0,4,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
2,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,1,2,1,2,4,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
4,1,2,1,1,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
5,1,2,1,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,1,2,1,3,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
7,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
8,1,2,1,1,3,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
9,1,2,1,1,3,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
10,1,2,1,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [3]:
#Seleccionamos los atributos 
atributos = evaluation.loc[:, 'class':'Q27']
atributos = atributos.to_numpy()
scaler = preprocessing.MinMaxScaler()
atributos = scaler.fit_transform(atributos)
print(atributos[5818,:])

[1.   0.   0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.  ]


In [4]:
#Seleccionamos el objetivo
objetivo = evaluation['Q28']
objetivo = evaluation['Q28'].to_numpy(dtype=np.float32)
print(objetivo)

[3. 3. 5. ... 5. 1. 1.]


In [5]:
#Separamos los datos que se usaran para entrenar y los que se usarán para evaluar los modelos
(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = model_selection.train_test_split(
    atributos, objetivo, test_size=.15)

In [6]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(atributos_entrenamiento)
red_evaluation = keras.Sequential()#Creamos la red neuronal
red_evaluation.add(keras.layers.Input(shape=(31,)))#Definimos la capa de entrada con una neurona por cada atributo
red_evaluation.add(normalizador)#Aplicamos el normalizador
red_evaluation.add(keras.layers.Dense(70, activation='relu'))#Definimos una capa con 70 neuronas y función de activación relu
red_evaluation.add(keras.layers.Dense(70, activation='sigmoid'))#Definimos una capa de 70 neuronas y función de activación sigmoide
red_evaluation.add(keras.layers.Dense(1, activation='linear')) #Defnimios una capa de salida con función de activación lienal ya que los valores esperados son números lineales entre 0 y 5

In [7]:
red_evaluation.summary()#Mostramos un resumen de la red neuronal

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 31)               63        
 n)                                                              
                                                                 
 dense (Dense)               (None, 70)                2240      
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 1)                 71        
                                                                 
Total params: 7,344
Trainable params: 7,281
Non-trainable params: 63
_________________________________________________________________


In [8]:

red_evaluation.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])#Compilamos la red neuronal definiendo el optimizaodr, la función de perdida y dos metricas
red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)#Entrenamos la red neuronal con 10 epocas y lotes de 256 elementos

Epoch 1/10
20/20 [==============================] - 1s 5ms/step - loss: 2.0086 - accuracy: 0.1391 - mae: 1.0738
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.5204 - accuracy: 0.1417 - mae: 0.5208
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3553 - accuracy: 0.1417 - mae: 0.3859
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.3165 - accuracy: 0.1417 - mae: 0.3473
Epoch 5/10
20/20 [==============================] - 0s 5ms/step - loss: 0.3002 - accuracy: 0.1417 - mae: 0.3273
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.2895 - accuracy: 0.1417 - mae: 0.3191
Epoch 7/10
20/20 [==============================] - 0s 5ms/step - loss: 0.2812 - accuracy: 0.1417 - mae: 0.3097
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.2743 - accuracy: 0.1417 - mae: 0.3028
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.2688 - accuracy: 0.1417 - mae:

In [9]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
#Para este conjunto de datos esto practicamente no es necesario pero así nos aseguramos de cometer un error muy pequeño
perdObj = 1.0
x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
while x[0]>perdObj:
    red_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento,
                batch_size=256, epochs=10)
    x=red_evaluation.evaluate(atributos_prueba, objetivo_prueba)
    
print(x[0])

28/28 [==============================] - 0s 3ms/step - loss: 0.3243 - accuracy: 0.1271 - mae: 0.3210
0.3242553472518921


In [10]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_evaluation.evaluate(atributos_prueba, objetivo_prueba)

28/28 [==============================] - 0s 3ms/step - loss: 0.3243 - accuracy: 0.1271 - mae: 0.3210


[0.3242553472518921, 0.12714776396751404, 0.32097163796424866]

In [11]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
X=atributos[5809]
predicciones = red_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos[5810]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5811]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5812]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5813]
predicciones = red_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos[5814]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5815]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5816]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5817]
predicciones = red_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos[5818]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos[5819]
predicciones = red_evaluation.predict(X)
print('Esperado 1:',predicciones)

1/1 [==============================] - 0s 217ms/step
Esperado 4: [[4.1635756]]
1/1 [==============================] - 0s 50ms/step
Esperado 3: [[2.9934316]]
1/1 [==============================] - 0s 33ms/step
Esperado 3: [[3.0887675]]
1/1 [==============================] - 0s 39ms/step
Esperado 1: [[1.0869037]]
1/1 [==============================] - 0s 49ms/step
Esperado 3: [[3.0701988]]
1/1 [==============================] - 0s 31ms/step
Esperado 1: [[1.2919029]]
1/1 [==============================] - 0s 51ms/step
Esperado 1: [[1.0869037]]
1/1 [==============================] - 0s 56ms/step
Esperado 5: [[4.651767]]
1/1 [==============================] - 0s 51ms/step
Esperado 5: [[4.945514]]
1/1 [==============================] - 0s 50ms/step
Esperado 1: [[1.1100278]]
1/1 [==============================] - 0s 28ms/step
Esperado 1: [[1.1100278]]


In [12]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_evaluation = RandomForestRegressor() #Creamos el modelo
forest_evaluation.fit(atributos_entrenamiento, objetivo_entrenamiento)#Lo entrenamos
evaluaciones = forest_evaluation.predict(atributos_prueba)#Evaluamos
mse = mean_squared_error(objetivo_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)
#Como este modelo suele tener una perdida muy pequeña tras entrenarlo una vez, no hacemos nada más para mejorarlo

Error cuadrático medio (MSE) del modelo: 0.29200438584356353


In [13]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
atributos2 = atributos[:, np.newaxis]
X=atributos2[5809]
predicciones = forest_evaluation.predict(X)
print('Esperado 4:',predicciones)
X=atributos2[5810]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5811]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5812]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5813]
predicciones = forest_evaluation.predict(X)
print('Esperado 3:',predicciones)
X=atributos2[5814]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5815]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5816]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5817]
predicciones = forest_evaluation.predict(X)
print('Esperado 5:',predicciones)
X=atributos2[5818]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)
X=atributos2[5819]
predicciones = forest_evaluation.predict(X)
print('Esperado 1:',predicciones)

Esperado 4: [4.]
Esperado 3: [3.]
Esperado 3: [3.]
Esperado 1: [1.]
Esperado 3: [3.]
Esperado 1: [1.02]
Esperado 1: [1.]
Esperado 5: [4.95]
Esperado 5: [5.]
Esperado 1: [1.]
Esperado 1: [1.]


In [14]:
#Importamos las librerias necesarias para crear otra red neuronal
#Volemos a importar por si se quiere ejecutar este fragmento sin ejecutar el anterior
import pandas
import numpy as np
from sklearn import model_selection, preprocessing
from tensorflow import keras
import tensorflow as tf

#Leemos el csv
#bikes = pandas.read_csv('hour.csv', header=0)
bikes = pandas.read_csv('hour2.csv',header = 0)


In [15]:
#Mostramos las 10 primeras lineas para ver que se ha leido bién
bikes.head(10)

,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0000,3,13,16
1,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0000,8,32,40
2,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0000,5,27,32
3,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0000,3,10,13
4,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0000,0,1,1
5,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
6,1,0,1,6,0,6,0,1,0.22,0.2727,0.80,0.0000,2,0,2
7,1,0,1,7,0,6,0,1,0.20,0.2576,0.86,0.0000,1,2,3
8,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0.0000,1,7,8
9,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0.0000,8,6,14


In [16]:
#Seleccionamos los atributos y los nomalizamos
attr = bikes.loc[:, 'season':'registered']
attr = attr.to_numpy()
scaler = preprocessing.MinMaxScaler()
attr = scaler.fit_transform(attr)

print(attr)

[[0.         0.         0.         ... 0.         0.00817439 0.01467269]
 [0.         0.         0.         ... 0.         0.02179837 0.03611738]
 [0.         0.         0.         ... 0.         0.01362398 0.03047404]
 ...
 [0.         1.         1.         ... 0.19301751 0.01907357 0.09367946]
 [0.         1.         1.         ... 0.15786999 0.03542234 0.05417607]
 [0.         1.         1.         ... 0.15786999 0.03269755 0.04176072]]


In [17]:
#Seleccionamos los objetivos
obj = bikes['cnt']
obj = bikes['cnt'].to_numpy(dtype=np.float32)
print(obj)

[16. 40. 32. ... 90. 61. 49.]


In [18]:
#Cogemos un 85% de los datos para entrenar y un 15% de los datos para evaluar el modelo
(attr_entrenamiento, attr_prueba,
 obj_entrenamiento, obj_prueba) = model_selection.train_test_split(
    attr, obj, test_size=.10)

In [19]:
normalizador = keras.layers.Normalization()#Definimos un normalizador
normalizador.adapt(attr_entrenamiento)
red_bikes = keras.Sequential()#Creamos la red neuronal
red_bikes.add(keras.layers.Input(shape=(14,))) #Añadimos la capa de entrada con 14 neuronas, una por cada atributo
red_bikes.add(normalizador)#Aplicamos un normalizador
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(100, activation='sigmoid'))#Añadimos una capa intermedia de 100 neuronas con función de activación sigmoide
red_bikes.add(keras.layers.Dense(1, activation='linear'))#Necesitamos una capa de salida linear ya que los valores objetivos son lineales

In [20]:
#Vemos un resumen de la red
red_bikes.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_1 (Normalizat  (None, 14)               29        
 ion)                                                            
                                                                 
 dense_3 (Dense)             (None, 100)               1500      
                                                                 
 dense_4 (Dense)             (None, 100)               10100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 11,730
Trainable params: 11,701
Non-trainable params: 29
_________________________________________________________________


In [21]:
#Compilamos y entrenamos la red neuronal, buscamos minimizar la función de perdida
red_bikes.compile(optimizer='SGD', loss='mean_squared_error', metrics=['accuracy', 'mae'])
red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)

Epoch 1/10
62/62 [==============================] - 1s 5ms/step - loss: 10915.7881 - accuracy: 0.0091 - mae: 67.1878
Epoch 2/10
62/62 [==============================] - 0s 5ms/step - loss: 1514.4172 - accuracy: 0.0091 - mae: 27.5313
Epoch 3/10
62/62 [==============================] - 0s 5ms/step - loss: 610.2152 - accuracy: 0.0091 - mae: 16.7194
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 505.1913 - accuracy: 0.0091 - mae: 15.4094
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 219.1440 - accuracy: 0.0091 - mae: 9.5991
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 178.1140 - accuracy: 0.0091 - mae: 9.8253
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 214.9675 - accuracy: 0.0091 - mae: 9.5953
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 87.6014 - accuracy: 0.0091 - mae: 6.7033
Epoch 9/10
62/62 [==============================] - 0s 5ms/step - loss: 135.3069 -

In [22]:
#Definimos cuanta perdida queremos tener y ejecutamos el siguiente bucle para entrenar la red hasta conseguir una perdida menor a la definida
perdidaObj = 1.0
x=red_bikes.evaluate(attr_prueba, obj_prueba)
while x[0]>perdidaObj:
    red_bikes.fit(attr_entrenamiento, obj_entrenamiento,
                batch_size=256, epochs=10)
    x=red_bikes.evaluate(attr_prueba, obj_prueba)
    
#La forma más efectiva que hemos encontrado para mejorar nuestra red ha sido la de hacer este bulce ya que es la unica forma de asegurarnos que la red obtendrá la precisión que busquemos
print(x[0])

55/55 [==============================] - 0s 3ms/step - loss: 121.9681 - accuracy: 0.0086 - mae: 8.3876
Epoch 1/10
62/62 [==============================] - 0s 4ms/step - loss: 96.8324 - accuracy: 0.0091 - mae: 7.3383
Epoch 2/10
62/62 [==============================] - 0s 4ms/step - loss: 79.7620 - accuracy: 0.0091 - mae: 5.4665
Epoch 3/10
62/62 [==============================] - 0s 4ms/step - loss: 58.2903 - accuracy: 0.0091 - mae: 5.5287
Epoch 4/10
62/62 [==============================] - 0s 4ms/step - loss: 62.6145 - accuracy: 0.0091 - mae: 5.2642
Epoch 5/10
62/62 [==============================] - 0s 4ms/step - loss: 24.9311 - accuracy: 0.0091 - mae: 3.3661
Epoch 6/10
62/62 [==============================] - 0s 4ms/step - loss: 69.6575 - accuracy: 0.0091 - mae: 6.1104
Epoch 7/10
62/62 [==============================] - 0s 4ms/step - loss: 33.2056 - accuracy: 0.0091 - mae: 4.0961
Epoch 8/10
62/62 [==============================] - 0s 4ms/step - loss: 23.3364 - accuracy: 0.0091 - mae: 

62/62 [==============================] - 0s 5ms/step - loss: 1.1860 - accuracy: 0.0091 - mae: 0.7412
Epoch 8/10
62/62 [==============================] - 0s 5ms/step - loss: 20.0747 - accuracy: 0.0091 - mae: 3.3826
Epoch 9/10
62/62 [==============================] - 0s 5ms/step - loss: 5.4509 - accuracy: 0.0091 - mae: 1.6421
Epoch 10/10
55/55 [==============================] - 0s 3ms/step - loss: 3.2417 - accuracy: 0.0086 - mae: 0.9472
Epoch 1/10
62/62 [==============================] - 0s 5ms/step - loss: 1.0263 - accuracy: 0.0091 - mae: 0.6932
Epoch 2/10
62/62 [==============================] - 0s 5ms/step - loss: 2.8097 - accuracy: 0.0091 - mae: 1.1231
Epoch 3/10
62/62 [==============================] - 0s 5ms/step - loss: 1.2287 - accuracy: 0.0091 - mae: 0.7909
Epoch 4/10
62/62 [==============================] - 0s 5ms/step - loss: 2.6728 - accuracy: 0.0091 - mae: 1.1515
Epoch 5/10
62/62 [==============================] - 0s 5ms/step - loss: 1.5533 - accuracy: 0.0091 - mae: 0.8326
E

In [23]:
#Ahora que ya hemos conseguido una perdida menor a la que queriamos, evaluamos para volver a comprobar que la hemos conseguido
red_bikes.evaluate(attr_prueba, obj_prueba)

55/55 [==============================] - 0s 3ms/step - loss: 0.5764 - accuracy: 0.0086 - mae: 0.5460


[0.5763916373252869, 0.008630610071122646, 0.5459567904472351]

In [24]:
#Ahora mismo comprobaremos manualmente si las predicciones de la red se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado

X=attr[17368]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 203:',predicciones)
X=attr[17369]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 247:',predicciones)
X=attr[17370]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 315:',predicciones)
X=attr[17371]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 214:',predicciones)
X=attr[17372]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 164:',predicciones)
X=attr[17373]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 122:',predicciones)
X=attr[17374]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 119:',predicciones)
X=attr[17375]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 89:',predicciones)
X=attr[17376]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 90:',predicciones)
X=attr[17377]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 61:',predicciones)
X=attr[17378]
predicciones = red_bikes.predict(X,verbose=0)
print('Esperado 49:',predicciones)

Esperado 203: [[202.2305]]
Esperado 247: [[247.01466]]
Esperado 315: [[315.46652]]
Esperado 214: [[213.33109]]
Esperado 164: [[164.65822]]
Esperado 122: [[123.29248]]
Esperado 119: [[120.30547]]
Esperado 89: [[89.25454]]
Esperado 90: [[89.97709]]
Esperado 61: [[60.226505]]
Esperado 49: [[49.236687]]


In [25]:
#Ahora trabajaremos con el otro modelo que será un random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
forest_bikes = RandomForestRegressor()#Creamos el randomForest
forest_bikes.fit(attr_entrenamiento, obj_entrenamiento)#Lo entrenamos
evaluaciones = forest_bikes.predict(attr_prueba)#Evaluamos el modelo
mse = mean_squared_error(obj_prueba, evaluaciones)#Calculamos la perdida
print("Error cuadrático medio (MSE) del modelo:", mse)

Error cuadrático medio (MSE) del modelo: 3.149963866513232


In [26]:
#Ahora mismo comprobaremos manualmente si las predicciones del random forest se parecen a lo que esperabamos utilizando los ultimos registros con los cuales no se ha entrenado
attr2 = attr[:, np.newaxis]
X=attr2[17368]
predicciones = forest_bikes.predict(X)
print('Esperado 203:',predicciones)
X=attr2[17369]
predicciones = forest_bikes.predict(X)
print('Esperado 247:',predicciones)
X=attr2[17370]
predicciones = forest_bikes.predict(X)
print('Esperado 315:',predicciones)
X=attr2[17371]
predicciones = forest_bikes.predict(X)
print('Esperado 214:',predicciones)
X=attr2[17372]
predicciones = forest_bikes.predict(X)
print('Esperado 164:',predicciones)
X=attr2[17373]
predicciones = forest_bikes.predict(X)
print('Esperado 122:',predicciones)
X=attr2[17374]
predicciones = forest_bikes.predict(X)
print('Esperado 119:',predicciones)
X=attr2[17375]
predicciones = forest_bikes.predict(X)
print('Esperado 89:',predicciones)
X=attr2[17376]
predicciones = forest_bikes.predict(X)
print('Esperado 90:',predicciones)
X=attr2[17377]
predicciones = forest_bikes.predict(X)
print('Esperado 61:',predicciones)
X=attr2[17378]
predicciones = forest_bikes.predict(X)
print('Esperado 49:',predicciones)

Esperado 203: [202.75]
Esperado 247: [246.89]
Esperado 315: [314.87]
Esperado 214: [214.25]
Esperado 164: [163.86]
Esperado 122: [122.22]
Esperado 119: [118.93]
Esperado 89: [89.06]
Esperado 90: [90.07]
Esperado 61: [61.08]
Esperado 49: [49.03]


In [27]:
#Metodo LIME
import random
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPClassifier, MLPRegressor



"""
Partiremos de este seudocódigo para implementar el metodo
N es el n´umero de permutaciones a realizar
f es el modelo a explicar
X′ ← {} muestras perturbadas
R ← {} representaciones
W ← {} las distancias entre la muestra x y sus perturbaciones
for 1
to
N do
Selecciona k atributos aleatoriamente
x′ ← una perturbaci´on de x donde se perturban los k atributos anteriores.
w ← la distancia entre x y x′
r ← la representaci´on de x′ respecto a x
X′ ← X′ ∪ x′
R ← R ∪ r
W ← R ∪ w
end for
Y ′ ← f(X′) las predicciones de las perturbaciones
G ← modelo ridge entrenado con R para predecir Y ′ y ponderando cada
muestra con W
return los par´ametros de G

"""

def explain_model(f, x, N,M):
    #f es el modelo a explicar
    #X es una lista de ejemplos a los que se aplicará LIME
    #N es el número de permutaciones a realizar
    #M contiene todos los ejemplos
    #Si queremos que el algoritmo cumpla la metrica identidad necesitamos fijar semillas para la aleatoriedad
    random.seed(11)
    Xi = [] #Aqui guardaremos las muestras perturbadas
    R = []  #Aqui guardaremos las representaciones
    W = []  #Aqui guradaremos las distancias
    for i in range(N):
        k = random.randint(1, len(x)) #Escojo un número aletario que representa el número de los atributos a seleccionar
        perturbed_x = x.copy() #copio la muestra original
        for j in range(k): 
            perturbed_attr = random.randint(0,len(x)-1)#Voy escojiento los atributos que perturbare de forma aleatoria
            mx = abs(max([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor máximo
            mn = abs(min([a[perturbed_attr] for a in M])) #Para el atributo aletorio seleccionado calculo el valor mínimo
            perturbed_x[perturbed_attr] = random.uniform(mn, mx) # pertubo el atributo con un valor aleatorio ente mn y mx para que el valor esté acotado
        w = abs(sum([x[attr] - perturbed_x[attr] for attr in range(0,len(x)-1)])) #Calculo la distancia entre el original y el perturbado
        r = [0  if perturbed_x[attr] == x[attr] else 1 for attr in range(0,len(x)-1)] #Calculo la representación del perturbado respecto al original utilizando un operador ternario
        Xi.append(perturbed_x) #Acumulo en la lista los perturbados
        R.append(r)#Acumulamos en la lista de representaciones
        W.append(w)#Añadimos las distancias a la lista
    Y_perturbed = []#Aqui guardaremos las predicciones de las perturbaciones
    for i in range(len(Xi)):
        y = 1
            #En los siguiente if y else comprobamos el tipo de modelo que tenemos ya que dependiendo del tipo necesita recibir los atributos de una forma u otra
        if isinstance(f, keras.Sequential):#Comprueba si el modelo es una red neuronal
            xi = Xi[i]
            y = f.predict(xi,verbose=0)#Hacemos la prediccion con verbose=0 para no cargar la salida de lineas generadas por keras
        else: #Si no, es un randomForest
            xi = Xi[i]
            array = np.array(xi)
            xi = array.reshape(1, -1)
            y = f.predict(xi)#RandomForest no tiene verbose porque no genera nada
                
            
        Y_perturbed.append(y) #Aplico el modelo f a los ejemplos perturbados
            
    Y_perturbed = np.squeeze(Y_perturbed) #Por alguna razón aparece el error Found array with dim 3. Estimator expected <= 2" y lo arreglamos con esta linea que quita una dimensión al array
    G = Ridge()
    G.fit(R, Y_perturbed, sample_weight=W)
    return G.coef_#Devolvemos los parametros


In [28]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.02432406,  0.01243588,  0.01662844, -0.05516985,  0.00396146,
       -0.02734023, -0.04393756, -0.05084051, -0.04591982,  0.01510503,
        0.01737612, -0.01700965, -0.05885665,  0.00405327, -0.03006711,
       -0.07217951, -0.00999771, -0.02601384, -0.05599222,  0.03124656,
       -0.01269099, -0.01405098, -0.0430235 ,  0.04410394, -0.04393756,
       -0.01205467, -0.02797656, -0.04434989, -0.04121068, -0.01719019])

In [29]:
#Hacemos varias pruebas para comprobar que funciona el metodo LIME
atributivos = atributos[5809]

explain_model(red_evaluation,atributivos,10,atributos)

array([-0.02432406,  0.01243588,  0.01662844, -0.05516985,  0.00396146,
       -0.02734023, -0.04393756, -0.05084051, -0.04591982,  0.01510503,
        0.01737612, -0.01700965, -0.05885665,  0.00405327, -0.03006711,
       -0.07217951, -0.00999771, -0.02601384, -0.05599222,  0.03124656,
       -0.01269099, -0.01405098, -0.0430235 ,  0.04410394, -0.04393756,
       -0.01205467, -0.02797656, -0.04434989, -0.04121068, -0.01719019])

In [30]:
atributivos = atributos[5809]

explain_model(forest_evaluation,atributivos,10,atributos)

array([-0.05329193,  0.05785083, -0.02938303, -0.00121754,  0.03118763,
        0.02368814, -0.0924245 , -0.04637594, -0.07885433, -0.12118314,
        0.06010089, -0.0124536 , -0.00507487,  0.05150461, -0.11055356,
       -0.01367114, -0.01184712, -0.05904895, -0.10843869,  0.04197182,
       -0.05045268, -0.06335174, -0.07150255,  0.02386253, -0.0924245 ,
       -0.0160142 , -0.01075033,  0.01123454,  0.04181721, -0.0939344 ])

In [31]:
attri = attr[17378]

explain_model(red_bikes,attri,10,attr)

array([ -14.41166898, -108.19534129,  183.00625567,   76.05185988,
         34.30216426, -164.87356029,  133.62825227,  -97.76374446,
         37.47019646,   -3.57918921,  -51.81973685,  -48.83394444,
        150.88655541])

In [32]:
attri = attr[17378]

explain_model(forest_bikes,attri,10,attr)

array([ -26.3872849 ,  -95.85683724,  190.29437754,   63.31487811,
         50.39551782, -160.89999858,  147.04929586, -111.50024267,
         20.8976087 ,  -18.79098559,  -52.05029761,  -64.21018778,
        127.66247407])

In [33]:
#En la definición de cada metrica se importarán las cosas necesarias por si solo se quiere probar una metrica
import numpy as np


def identidad(x1,x2,exp1,exp2):
    #Definición: El principio de identidad establece que objetos idénticos deben recibir explicaciones idénticas.

    res = True #Si los objetos no son identicos no hay que valuar
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance ==0):   #Solo comprobamos si las explicaciones sean identicas si los objetos son identicos
        res = np.linalg.norm(exp1 - exp2) ==0#Comprobamos si ambas explicaciones son iguales
    return res

In [34]:
ejemplo2=attr[17375]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(identidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [35]:
import numpy as np


def separabilidad(x1,x2,exp1,exp2):
#Definición:#Separabilidad: Objetos no idénticos no pueden tener explicaciones idénticas. Para simplificar, cada característica 
#tiene un nivel mínimo de importancia, positivo o negativo, en las predicciones.

    res = True
    distance = np.linalg.norm(x1 - x2)#Calcula las distancias entre los dos ejemplos
    if(distance >0):   #Si la distancia es distinta de cero
        res = not(np.linalg.norm(exp1 - exp2) ==0)#Si las explicaciones son distintas se cumple la separabilidad
    
    return res

In [36]:
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(separabilidad(ejemplo1,ejemplo2,exp1,exp2))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
True


In [37]:
import numpy as np
from scipy.stats import spearmanr

def estabilidad(x1,x2,exp1,exp2,umbEj,umbExp):
    #Objetos similares deben tener explicaciones similares
    #Para esto calcularemos la correlacion de Spearman para los ejemplos, si superan el limite que se pasará a la función
    #Calcularemos la correlación de Spearman para las explicaciones y comprobaremos si supera el umbral, en ese caso, cumpliran la estabilidad
    res = True
    correlacionEj, _ = spearmanr(x1, x2)
    if(abs(correlacionEj) >= umbEj):
        correlacionExp, _ = spearmanr(exp1,exp2)
        res = abs(correlacionExp)>=umbExp 
    return res

In [38]:
ejemplo2=attr[17376]
ejemplo1 = attr[17375]
print(ejemplo2)
print(ejemplo1)
print(spearmanr(ejemplo1, ejemplo2))
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
print(spearmanr(exp1, exp2))
print(estabilidad(ejemplo1,ejemplo2,exp1,exp2,0.75,0.75))

[0.         1.         1.         0.91304348 0.         0.16666667
 1.         0.         0.24489796 0.2576     0.6        0.19301751
 0.01907357 0.09367946]
[0.         1.         1.         0.86956522 0.         0.16666667
 1.         0.33333333 0.24489796 0.2576     0.6        0.19301751
 0.02179837 0.09142212]
SpearmanrResult(correlation=0.8770812523243259, pvalue=3.801615077865054e-05)
SpearmanrResult(correlation=1.0, pvalue=0.0)
True


In [45]:
def coherencia(mae1, mae2):
    #Se calcula la diferencia entre el error de predicción (mae1) sobre la señal original y el error de predicción mae2
    #de una nueva señaal donde se eliminan las características no importantes.
    alpha_i = np.abs(mae1 - mae2)
    return alpha_i


In [60]:
from sklearn.metrics import mean_absolute_error

evaluaciones1 = red_bikes.predict(attr[17375],verbose=0)
evaluaciones2 = red_bikes.predict(attr[17376],verbose=0)
mae1 = mean_absolute_error(evaluaciones1,evaluaciones2)


evaluaciones3 = explain_model(red_bikes,attr[17375],10,attr)
evaluaciones4 = explain_model(red_bikes,attr[17376],10,attr)
mae2 = mean_absolute_error(evaluaciones3,evaluaciones4)

coherencia1 = coherencia(mae1,mae2)
print(coherencia1)

evaluaciones5 = red_bikes.predict(attr[17373],verbose=0)
evaluaciones6 = red_bikes.predict(attr[17374],verbose=0)
mae3 = mean_absolute_error(evaluaciones5,evaluaciones6)


evaluaciones7 = explain_model(red_bikes,attr[17373],10,attr)
evaluaciones8 = explain_model(red_bikes,attr[17374],10,attr)
mae4 = mean_absolute_error(evaluaciones7,evaluaciones8)

coherencia2 = coherencia(mae3,mae4)
print(coherencia2)

7.298621440894193
20.271639057122528


In [55]:
import numpy as np

def completitud(x, exp):
    
    # Calculamos los pesos de las instancias de datos y sus respectivas explicaciones LIME
    data_weights = np.exp(-np.linalg.norm(x - x.mean(axis=0), axis=0))
    exp_weights = np.exp(-np.linalg.norm(exp - exp.mean(axis=0), axis=0))

    # Calculamos los valores de ei y pi
    ei = np.sum(exp_weights)
    pi = np.sum(data_weights)

    # Calculamos la métrica Completitud
    gamma = ei / pi

    # Imprimimos el valor de la métrica Completitud
    return print("El valor de la métrica Completitud es:", gamma)

In [56]:
ejemplo1 = attr[17375]
exp1 = explain_model(forest_bikes,ejemplo1,2,attr)
completitud(ejemplo1,exp1)

El valor de la métrica Completitud es: 4.7577439823703274e-57


In [61]:
import numpy as np

def congruencia(alpha_values):
    """
    Calcula la métrica de Congruencia para el método LIME.

    Args:
        alpha_values (ndarray): Arreglo de valores de alpha calculados para cada característica.

    Returns:
        float: Valor de la métrica de Congruencia.
        
    La desviación estándar de la coherencia proporciona el
    proxy de congruencia. Esta métrica ayuda a capturar la variabilidad de
    la coherencia.
    """

    alpha_avg = np.mean(alpha_values)  # Coherencia promedio
    n = alpha_values.shape[0]  # Número de características

    # Calcula la métrica de Congruencia
    delta = np.sum(np.square(alpha_values - alpha_avg)) / n

    return delta


In [63]:
valores_coherencia = np.array((coherencia1,coherencia2))
congruencia(valores_coherencia)

42.07479651774267

In [ ]:
#METODO CONGRUENCIA
# Creamos una lista de valores de coherencia para un conjunto de N muestras
alpha_values = [0.8, 0.6, 0.7, 0.9, 0.5]

# Calculamos el valor promedio de la coherencia
alpha = np.mean(alpha_values)

# Creamos una lista de valores de alpha_i para cada instancia de datos
alpha_i_values = [0.9, 0.7, 0.6, 0.8, 0.4]

# Calculamos el número de instancias de datos
N = len(alpha_i_values)

# Calculamos la suma de las diferencias cuadráticas entre alpha_i y alpha
diff_squares_sum = np.sum((np.array(alpha_i_values) - alpha) ** 2)

# Calculamos la métrica Congruencia
delta = np.sqrt((diff_squares_sum / N) / alpha)

# Imprimimos el valor de la métrica Congruencia
print("El valor de la métrica Congruencia es:", delta)


In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score

def selectividad(X, y, model):
    """
    Calcula la métrica de Selectividad para el método LIME.

    Args:
        X (ndarray): Matriz de instancias de datos de forma (n, m), donde n es el número de instancias y m es el número de características.
        y (ndarray): Vector de etiquetas de forma (n,), donde n es el número de instancias.
        model: Modelo de aprendizaje automático ajustado sobre los datos completos.

    Returns:
        float: Área bajo la curva (AUC) que mide la selectividad al eliminar características una por una.
    """

    n = X.shape[0]  # número de instancias
    m = X.shape[1]  # número de características

    auc_scores = []  # lista para almacenar los resultados del AUC

    # Realiza las predicciones sobre los datos completos
    y_pred_base = model.predict_proba(X)[:, 1]

    # Calcula el AUC para el modelo sobre los datos completos
    auc_base = roc_auc_score(y, y_pred_base)

    # Itera sobre cada característica y calcula el AUC al eliminarla
    for i in range(m):
        X_copy = X.copy()
        X_copy[:, i] = 0  # Establece en cero la i-ésima característica

        # Realiza las predicciones con la característica eliminada
        y_pred = model.predict_proba(X_copy)[:, 1]

        # Calcula el AUC y lo agrega a la lista de resultados
        auc_score = roc_auc_score(y, y_pred)
        auc_scores.append(auc_score)

    # Calcula el AUC promedio para la selectividad
    auc_selectivity = np.mean(auc_scores)

    return auc_selectivity


In [ ]:
#METODO SELECTIVIDAD
# Cargamos el dataset de cáncer de mama de scikit-learn
data = load_breast_cancer()

# Definimos las características y la variable objetivo
X = data['data']
y = data['target']

# Creamos un modelo de regresión logística
model = LogisticRegression()

# Entrenamos el modelo
model.fit(X, y)

# Obtenemos las predicciones del modelo en el conjunto de datos completo
y_pred = model.predict_proba(X)[:, 1]

# Calculamos el AUC para el modelo entrenado en el conjunto de datos completo
auc_full = roc_auc_score(y, y_pred)

# Ordenamos las características de más a menos relevantes según el modelo
idx_sorted = np.argsort(np.abs(model.coef_))[0][::-1]

# Creamos una lista para almacenar los AUC para cada característica eliminada
auc_scores = []

# Iteramos sobre las características de más a menos relevantes según el modelo
for i in range(X.shape[1]):
    # Establecemos la característica i en cero
    X_test = X.copy()
    X_test[:, idx_sorted[i]] = 0
    
    # Obtenemos las predicciones del modelo en el conjunto de datos con la característica i eliminada
    y_pred_test = model.predict_proba(X_test)[:, 1]
    
    # Calculamos el AUC para el modelo entrenado en el conjunto de datos con la característica i eliminada
    auc_test = roc_auc_score(y, y_pred_test)
    
    # Almacenamos el AUC para la característica i eliminada en la lista de AUC
    auc_scores.append(auc_test)

# Calculamos la métrica Selectividad como el área bajo la curva ROC para el gráfico de AUC vs características eliminadas
selectivity = np.trapz(auc_scores, dx=1)

# Imprimimos el valor de la métrica Selectividad
print("El valor de la métrica Selectividad es:", selectivity)


In [43]:
#Comprobaremos la métrica identidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y para cada ejemplo en cada modelo generaremos la explicacion dos veces y comprobaremos que es la misma explicación
atributivos = atributos[5553:5809,:]
sonIguales = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
if(sonIguales):
    print("El modelo RandomForest de evaluación cumple la métrica identidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica identidad")

sonIguales=True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,x,2,atributos)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break

if(sonIguales):
    print("El modelo Red neuronal de evaluación cumple la métrica identidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica identidad")

X=attr[17122:17378,:]
sonIguales=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo Red neuronal de bikes cumple la métrica identidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica identidad")
    
sonIguales=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,x,2,attr)
    sonIguales = sonIguales and identidad(x,x,exp1,exp2)
    if(not sonIguales):
        break
    
if(sonIguales):
    print("El modelo RandomForest de bikes cumple la métrica identidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica identidad")


El modelo RandomForest de evaluación cumple la metrica identidad
El modelo Red neuronal de evaluación cumple la metrica identidad
El modelo Red neuronal de bikes cumple la metrica identidad
El modelo RandomForest de bikes cumple la metrica identidad


In [44]:
#Comprobaremos la métrica separabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo
ejemplo1=atributos[5552]
atributivos = atributos[5553:5809,:]
cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
if(cumpleSeparabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleSeparabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo1,exp1,exp2)
    if(not cumpleSeparabilidad):
        break

if(cumpleSeparabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17121]
X=attr[17122:17378,:]
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica separabilidad")
    
cumpleSeparabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleSeparabilidad=cumpleSeparabilidad and separabilidad(x,ejemplo2,exp1,exp2)
    if(not cumpleSeparabilidad):
        break
    
if(cumpleSeparabilidad):
    print("El modelo RandomForest de bikes cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica separabilidad")


El modelo RandomForest de evaluación cumple la metrica separabilidad
El modelo Red neuronal de evaluación cumple la metrica separabilidad
El modelo Red neuronal de bikes cumple la metrica separabilidad
El modelo RandomForest de bikes cumple la metrica separabilidad


In [59]:
#Comprobaremos la métrica estabilidad
#Para comprobarla recorreremos 256 ejemplos en los cuatro modelos y los comparamos con otro ejemplo, del cual sabemos que hay ejemplos muy similares

ejemplo1=atributos[5819]
atributivos = atributos[5553:5819,:]
cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(forest_evaluation,x,2,atributos)
    exp2 = explain_model(forest_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
if(cumpleEstabilidad):
    print("El modelo RandomForest de evaluación cumple la métrica separabilidad")
else:
    print("El modelo RandomForest de evaluación no cumple la métrica separabilidad")

cumpleEstabilidad = True
for x in atributivos:
    exp1 = explain_model(red_evaluation,x,2,atributos)
    exp2 = explain_model(red_evaluation,ejemplo1,2,atributos)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo1,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break

if(cumpleEstabilidad):
    print("El modelo Red neuronal de evaluación cumple la métrica separabilidad")
else:
    print("El modelo Red neuronal de evaluación no cumple la métrica separabilidad")

ejemplo2 = attr[17376]
X=attr[17122:17378,:]
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(red_bikes,x,2,attr)
    exp2 = explain_model(red_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo Red neuronal de bikes cumple la métrica estabilidad")
else:
    print("El modelo Red neuronal de bikes no cumple la métrica estabilidad")
    
cumpleEstabilidad=True
for x in X:
    exp1 = explain_model(forest_bikes,x,2,attr)
    exp2 = explain_model(forest_bikes,ejemplo2,2,attr)
    cumpleEstabilidad=cumpleEstabilidad and estabilidad(x,ejemplo2,exp1,exp2,0.75,0.75)
    if(not cumpleEstabilidad):
        break
    
if(cumpleEstabilidad):
    print("El modelo RandomForest de bikes cumple la métrica estabilidad")
else:
    print("El modelo RandomForest de bikes no cumple la métrica estabilidad")


El modelo RandomForest de evaluación cumple la métrica separabilidad
El modelo Red neuronal de evaluación cumple la métrica separabilidad
El modelo Red neuronal de bikes cumple la métrica estabilidad
El modelo RandomForest de bikes cumple la métrica estabilidad
